### Import packages

In [1]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# data structure
import pandas as pd

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

### Define all Variables

In [2]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
T_air = 298.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 8e-2 # W/m2/K
U_V = 8e-2 # W/m2/K

# Specify wall heat partitioning fraction
eta_w= 0.9

# Specify tank operating pressure
P = 100000 # Pa

### Variables of interest

# Vertically orientated cylindrical tank volume
V_tank = 165000 #m^3 | 

# Initial liquid filling / Dimensionless
LFs = [0.3, 0.95] # values of 0.95 and 0.30

# especify compound in the tank
Compound = "ammonia" # hydrogen, methane, ammonia, nitrogen

# Especify if the data should be saved in the output.xlxs excel
sv = True

In [3]:
# Define evaporation time to simulate
hours = [24 * 30, 24 * 2] #[24 * 2, 24 * 30]
evap_times = [x * 3600 for x in hours]
# helpful variables
mults = [1 , 2 , 3]
# Thickness of the in % of the internal diameter
ST = 1.02


# Show sim duration
print(f"Simulation time: {hours} h")

Simulation time: [720, 48] h


In [4]:
# Define objective function
def BOR_function(a):
    # Update aspect ratio
    # Calculate internal diameter
    d_i = ((4 * V_tank)/(np.pi * a))**(1/3) # internal diameter / m
    d_o = d_i * ST # external diameter / m
    
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    # Set cryogen
    mid_tank.cryogen = Cryogen(name = Compound)
    
    mid_tank.cryogen.set_coolprops(P)

    # Heat flux
    q_b = mult*U_L*(T_air-mid_tank.cryogen.T_sat)  
    
    Q_b=q_b*mid_tank.A_T
    
    #Set Heat transfer propierties
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Define vertical spacing
    # dz = 0.05 # 0.1 0.01 0.001
    # Calculate number of nodes
    
    # n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

    n_z = 50
    
    # Define dimensionless computational grid
    mid_tank.z_grid = np.linspace(0, 1, n_z)
    
    # Execute simulation
    mid_tank.evaporate(evap_time)
    
    # Calculate BOR
    BOR = (1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0])*(86400/mid_tank.data["Time"][-1])

    print("a = %.3f, BOR=%.3e" % (a, BOR))
    
    return BOR

In [5]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of the aspect ratio
bounds = Bounds([0.1], [5])

# Initial Aspect Ratio to optimise
x0 = 1 

# Solution of the optimization 
for mult in mults:
    for LF in LFs:
        for evap_time in evap_times:
            print(f"\nOptimum search for Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}, time: {evap_time/3600} hrs")
            res = minimize(BOR_function, x0, method='trust-constr',tol=1e-8 , options={'verbose': 1}, bounds=bounds)
            
            # Calculate internal diameter for optimum
            d_i = ((4 * V_tank)/(np.pi * res.x[0]))**(1/3) # internal diameter / m
            d_o = d_i * ST # external diameter / m
            
            # Initialize optimum tank
            mid_tank = Tank(d_i, d_o, V_tank, LF)
            
            # Set cryogen
            mid_tank.cryogen = Cryogen(name = Compound)
            
            mid_tank.cryogen.set_coolprops(P)
            
            q_b = mult*U_L*(T_air-mid_tank.cryogen.T_sat) # Heat flux proportional to the T diff of the cryogen.
            
            Q_b=q_b*mid_tank.A_T
            
            mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)
            
            mid_tank.U_roof = U_V
            
            # Data table
            
            table = [[Compound, mid_tank.V, mid_tank.LF, f"{evap_time/3600} hrs", f"{res.fun*100}", f"{res.x[0]}",
                        P, T_air, mid_tank.d_i, mid_tank.d_o, mid_tank.l, U_L, U_V, eta_w, mid_tank.Q_b, q_b]]
            
            # structure the data table
            
            data_table = pd.DataFrame(table, columns = ["Cryogen", "Volume", "Liquid Filling","Simulation","BOR","Aspect Ratio"
                                                    , "Pressure (P)", "T_air (K)", "d_i", "d_o", "Height", "U_L", "U_V", "Eta_w","Q_b","q_b"])
            #print(Data_table)
            
            if sv:
                # Send Data table to excel
                # Try to load the existing Excel file
                try:
                    data_excel = pd.read_excel('Final.xlsx')
                    
                    # Append the row
                    data_excel = pd.concat([data_excel, data_table], ignore_index = True)
                    
                    # Save the updated DataFrame back to the Excel file
                    data_excel.to_excel('Final.xlsx', index = False)
                except:
                    #In case the file doesnt exist it is created
                    data_table.to_excel('Final.xlsx', index = False)
                print("\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\nData was saved.\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
            else:
                print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\nData was not saved.\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
                # Print a summary of the optimum if it is found
            if res.success:
                print(f"\nOptimum found for Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}" + 
                      f"\nAspect Ratio = {res.x[0]:.3f}" + f"\nBOR = {res.fun*100:.3f} %")


Optimum search for Tank Volume: 165000, Liquid Filling: 30.0%, Compound: ammonia, time: 720.0 hrs


/tmp/ipykernel_30122/974823457.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=3.120e-03
a = 1.000, BOR=3.120e-03
a = 1.067, BOR=3.134e-03
a = 1.067, BOR=3.134e-03
a = 0.598, BOR=3.089e-03
a = 0.598, BOR=3.089e-03
a = 0.822, BOR=3.090e-03
a = 0.822, BOR=3.090e-03
a = 0.761, BOR=3.084e-03
a = 0.761, BOR=3.084e-03
a = 0.688, BOR=3.082e-03
a = 0.688, BOR=3.082e-03
a = 0.703, BOR=3.082e-03
a = 0.703, BOR=3.082e-03
a = 0.696, BOR=3.082e-03
a = 0.696, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
a = 0.694, BOR=3.082e-03
`gtol` termination condition is satisfied.
Number of iterations: 20, function evaluations: 22, CG iterations: 10, optimality: 3.88e-09, constraint violation: 0.00e+00, execution time: 1.6e+02 s.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Data was saved.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Optimum found for Tank Volume: 165000, Liquid Filling: 30.0%, Compound: ammonia
Aspect Ratio = 0.694
BOR = 0.308 %

Optimum search for Tank Volume: 165000, Liquid Filling: 30.

/tmp/ipykernel_30122/974823457.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=2.305e-04
a = 1.000, BOR=2.305e-04
a = 1.067, BOR=2.324e-04
a = 1.067, BOR=2.324e-04
a = 0.324, BOR=2.284e-04
a = 0.324, BOR=2.284e-04
a = 0.324, BOR=2.284e-04
a = 0.324, BOR=2.284e-04


/home/milcaito/anaconda3/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py:231: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


a = 0.930, BOR=2.286e-04
a = 0.930, BOR=2.286e-04
a = 0.832, BOR=2.262e-04
a = 0.832, BOR=2.262e-04
a = 0.490, BOR=2.220e-04
a = 0.490, BOR=2.220e-04
a = 0.664, BOR=2.228e-04
a = 0.664, BOR=2.228e-04
a = 0.584, BOR=2.219e-04
a = 0.584, BOR=2.219e-04
a = 0.545, BOR=2.218e-04
a = 0.545, BOR=2.218e-04
a = 0.540, BOR=2.218e-04
a = 0.540, BOR=2.218e-04
`gtol` termination condition is satisfied.
Number of iterations: 20, function evaluations: 22, CG iterations: 9, optimality: 9.79e-10, constraint violation: 0.00e+00, execution time: 4e+01 s.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Data was saved.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Optimum found for Tank Volume: 165000, Liquid Filling: 30.0%, Compound: ammonia
Aspect Ratio = 0.540
BOR = 0.022 %

Optimum search for Tank Volume: 165000, Liquid Filling: 95.0%, Compound: ammonia, time: 720.0 hrs


/tmp/ipykernel_30122/974823457.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("a = %.3f, BOR=%.3e" % (a, BOR))


a = 1.000, BOR=1.162e-03
a = 1.000, BOR=1.162e-03
a = 1.067, BOR=1.172e-03
a = 1.067, BOR=1.172e-03
a = 0.105, BOR=2.168e-03
a = 0.105, BOR=2.168e-03
a = 0.105, BOR=2.168e-03
a = 0.105, BOR=2.168e-03
a = 1.067, BOR=1.172e-03
a = 1.067, BOR=1.172e-03
a = 1.067, BOR=1.172e-03
a = 1.067, BOR=1.172e-03


KeyboardInterrupt: 